In [6]:
#########################
# makes bundle data
# collects energy, angles, flags, event ids
#########################

import numpy as np
import uproot as ur
import h5py as h5
import os

# cuts on number of signal hits 
cuts_signal = [8]
# output file name
h5_name = 'mc_baikal_bundle_pure_string-cut_v4_8OM_mu.h5'

# prefixes for files
particles = ['mu'] #,'nu'

# pathes in .root file to registered charges, times, and event flags.
pathes_data = ['Events/BEvent./BEvent.fPulses/BEvent.fPulses.fAmplitude', 'Events/BEvent./BEvent.fPulses/BEvent.fPulses.fTime', 
                'Events/MCEventMask./MCEventMask.BEventMask/MCEventMask.BEventMask.fOrigins/MCEventMask.BEventMask.fOrigins.fFlag']

# pathes to event id, angles, energy
pathes_ev_chars = ['Events/BJointHeader./BJointHeader.fEventID',
                'Events/BMCEvent./BMCEvent.fPrimaryParticlePolar', 'Events/BMCEvent./BMCEvent.fPrimaryParticleAzimuth',
                'Events/BMCEvent./BMCEvent.fPrimaryParticleEnergy']

# pathes to geometry data
pathes_geometry = 'ArrayConfig/BGeomTel./BGeomTel.BGeomTel/BGeomTel.BGeomTel.fOMs/BGeomTel.BGeomTel.fOMs.fPosition'

# dict for mapping string numbers to grid idxs
map_dict = {}
map_dict[6] = (0,0)
map_dict[0] = (0,1)
map_dict[1] = (0,2)
map_dict[5] = (1,0)
map_dict[7] = (1,1)
map_dict[2] = (1,2)
map_dict[4] = (2,0)
map_dict[3] = (2,1)

# routine for making bundle
def make_bundle(tr_channeles,data):
    # order data according to the registered charge
    # in this case for the channels appearing twice recorded is the time with largest signal
    sort_idxs = [ np.argsort(c,axis=0) for c in data[0] ]
    sorted_data = [ np.array([d[idx] for d,idx in zip(dt,sort_idxs)], dtype=object) for dt in data ]
    tr_channeles = np.array([d[idx] for d,idx in zip(tr_channeles,sort_idxs)], dtype=object) 
    # get grid idxs
    n_layer = [ [ ch % 36 for ch in channles ] for channles in tr_channeles ]
    n_string = [ [ ch // 36 for ch in channles ] for channles in tr_channeles ]
    n_grid = [ [ map_dict[n] for n in ns ] for ns in n_string ]
    coords = [ [ (z,x,y) for (z,(x,y)) in zip(zs,ss) ] for (zs,ss) in zip(n_layer,n_grid) ]
    # make bundle
    # data, mask, and labels seperatly
    bundle = np.zeros( (len(data[0]),36,3,3,2), dtype=np.float32 )
    mask_bundle = np.full( (len(data[0]),36,3,3), False )
    labels_bundle = np.zeros( (len(data[0]),36,3,3), dtype=np.float32 )
    for (m,l,b,chs,ts,fs,cs) in zip(mask_bundle,labels_bundle,bundle,sorted_data[0],sorted_data[1],sorted_data[2],coords):
        for (c,ch,t,f) in zip(cs,chs,ts,fs):
            m[c] = True
            l[c] = f
            b[c][0] = ch
            b[c][1] = t
    return (bundle, mask_bundle, labels_bundle)

# loop over particles -> files
for particle in particles:
    root_file_entries = os.scandir('/net/63/home/ivkhar/Baikal/data/MC/'+particle+'atm/') #/home/ivkhar/Baikal/data/MC/
    # loop over files
    file_num = 0
    for root_file in root_file_entries:
        id_prefix = root_file.name[:2]+'_'+root_file.name[-9:-5]+'_'
        with ur.open(root_file.path) as rf: 
            # read coordinates of the detectors
            # reading as awkward array and casting to numpy is faster for big arrays
            # yes it takes much memory during reading
            # in MC, only one entry
            coordinates_awk = rf[pathes_geometry].array( )
            coordinates = np.zeros((len(coordinates_awk[0]),3), dtype=np.float32)
            coordinates[:,0] = coordinates_awk['fX']
            coordinates[:,1] = coordinates_awk['fY']
            coordinates[:,2] = coordinates_awk['fZ']
            re_coordinates = np.reshape( coordinates, (36,8,3), order='F' )
            positions = np.zeros( (36,3,3,3), dtype=np.float32 )
            idxs = [map_dict[n] for n in range(8)]
            for p,cs in zip(positions,re_coordinates):
                for (idx,c) in zip(idxs,cs):
                    p[idx] = c
            # read data: channels, charges, times, flags
            data = [ rf[p].array(library="np") for p in pathes_data ]
            # get x, y, z as posiitons in cluster
            tr_channeles = rf['Events/BEvent./BEvent.fPulses/BEvent.fPulses.fChannelID'].array(library="np")
            # get idxs of signal hits
            flags = data[2]
            signals = [ np.nonzero(f!=0) for f in flags ]
            clear_data = [ [d[s] for (d,s) in zip(dd,signals)] for dd in data ]
            clear_tr_ch = [ d[s] for (d,s) in zip(tr_channeles,signals) ]
            # make bundle
            (bundle_data, bundle_mask, bundle_label)  = make_bundle(clear_tr_ch,clear_data)
            # get number of strings
            sig_channels = [ c[s] for c,s in zip(tr_channeles,signals) ]
            sig_strings = [ [c // 36 for c in ch] for ch in sig_channels ]
            un_strings = [ set(s) for s in sig_strings ]
            num_un_strings = np.array([ float(len(s)) for s in un_strings ])
            # get events characteristics          
            ev_chars = np.array([ rf[p].array(library="np") for p in pathes_ev_chars ])
            ev_chars = np.transpose( ev_chars, (1,0) )
            ev_chars = np.concatenate( [ev_chars,np.expand_dims(num_un_strings, axis=-1)], axis=-1 )
            #ev_chars = np.array([ ch for (ch,pp) in zip(ev_chars,pass_string_cut_events) if pp ])
            # impose cut on active channels and strings
            bi_labels = np.where( bundle_label!=0, 1, 0 )
            num_signal = np.sum( bi_labels, axis=(1,2,3) )
            for cut in cuts_signal:
                pass_idxs = np.nonzero( np.logical_and(num_signal>=cut,num_un_strings>1) )
                pass_bundle_data = bundle_data[pass_idxs]
                pass_bundle_label = bundle_label[pass_idxs]
                pass_bundle_mask = bundle_mask[pass_idxs]
                pass_bundle_ev_chars = ev_chars[pass_idxs][...,1:]
                pass_bundle_ev_ids = ev_chars[pass_idxs][...,0].tolist()
                pass_bundle_ev_ids = np.array([ id_prefix+str(int(ev_id)) for ev_id in pass_bundle_ev_ids ]).astype(np.string_)             
                # write data to file
                with h5.File('/home/leonov/Baikal/Cut_8_mu/Data/'+h5_name,'a') as hf:
                    hf.create_dataset('data/cut_'+str(cut)+'/'+particle+'/part_'+str(file_num)+'/data', data=pass_bundle_data, dtype='float32', compression='gzip', compression_opts=9)         
                    hf.create_dataset('labels/cut_'+str(cut)+'/'+particle+'/part_'+str(file_num)+'/data', data=pass_bundle_label, dtype='float32')
                    hf.create_dataset('ev_ids/cut_'+str(cut)+'/'+particle+'/part_'+str(file_num)+'/data', data=pass_bundle_ev_ids)
                    hf.create_dataset('ev_chars/cut_'+str(cut)+'/'+particle+'/part_'+str(file_num)+'/data', data=pass_bundle_ev_chars)
                    hf.create_dataset('mask/cut_'+str(cut)+'/'+particle+'/part_'+str(file_num)+'/data', data=pass_bundle_mask.astype(bool), dtype=bool)
            file_num += 1
# write coordinates
# assumed that they are fixed
with h5.File('/home/leonov/Baikal/Cut_8_mu/Data/'+h5_name,'a') as hf:
    hf.create_dataset('geometry/bundle_coordinates', data=positions, dtype='float32')
print(file_num)

1990


In [7]:
import numpy as np
import h5py as h5

cuts_signal = [8]
h5f = 'mc_baikal_bundle_pure_string-cut_v4_8OM_mu.h5'
use_postfix = False # True

particles = ['mu'] #'mu',
num_files_to_use = {'mu':1990} #'mu':0,
if use_postfix:
    postfix = '_mu-'+str(num_files_to_use['mu'])+'_nu-'+str(num_files_to_use['nu'])
else:
    postfix = ''

with h5.File('/home/leonov/Baikal/Cut_8_mu/Data/'+h5f,'a') as hf:
    for cut in cuts_signal:
        # get numbers of events
        nums_dict = {}
        parts_dict = {}
        for particle in particles:
            nums_dict[particle] = {}
            parts = list(hf['data/cut_'+str(cut)+'/'+particle].keys())
            parts_dict[particle] = parts[:num_files_to_use[particle]]    
            for part in parts_dict[particle]:
                nums_dict[particle][part] = hf['data/cut_'+str(cut)+'/'+particle+'/'+part+'/data'].shape[0]
        # get local avg
        means = []
        nums = []
        for particle in particles:
            for part in parts_dict[particle]:
                data = hf['data/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()]
                shape = data.shape[1:]
                axs = tuple( i for i in range(len(shape)) )
                mean = np.mean(data, axis=axs, dtype=np.float64)
                hf.create_dataset('data/cut_'+str(cut)+'/'+particle+'/'+part+'/norm_param/mean'+postfix, data=mean, dtype=np.float32)
                means.append(mean)
                nums.append(nums_dict[particle][part])
        # get global avg
        # calculating factor as this give better precision
        total = sum(nums)
        gl_mean = 0
        for m,n in zip(means,nums):
            factor = n/total
            gl_mean += m*factor
        hf.create_dataset('norm_param_global/cut_'+str(cut)+'/mean'+postfix, data=gl_mean)
        # get stds
        stds = []
        for particle in particles:
            for part in parts_dict[particle]:
                data = hf['data/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()]
                shape = data.shape[1:]
                axs = tuple( i for i in range(len(shape)) )
                std = np.sqrt( np.mean((data-gl_mean)**2, axis=axs, dtype=np.float64) )
                hf.create_dataset('data/cut_'+str(cut)+'/'+particle+'/'+part+'/norm_param/std'+postfix, data=std, dtype=np.float32)
                stds.append(std)
        # get global std
        # a lot of events and big stds, hence intoducing factor should be good
        gl_std = 0.
        for s,n in zip(stds,nums):
            factor = n/total
            gl_std += np.power(s,2)*factor
        gl_std = np.sqrt(gl_std)
        hf.create_dataset('norm_param_global/cut_'+str(cut)+'/std'+postfix, data=gl_std)


In [ ]:
import numpy as np
import h5py as h5
import random as rd
import uproot as ur
import os
h5_in = 'mc_baikal_bundle_pure_string-cut_v4_8OM_mu.h5'
h5_out_postfix = '_pure'
mean_posfix = ''
#mean_posfix = '_mu-'+X+'_nu-'+Y

dsets = ['train','test','val']
particles = ['mu'] #'mu',
cuts_signal = [8]

# number of files to use for test and val sets
# taking last files
num_files_to_test = { 'mu':30} #'mu':30,
num_files_to_val = { 'mu':30} #'mu':30,
num_files_to_train = { 'mu':1900} #'mu':50,
 
with h5.File('/home/leonov/Baikal/Cut_8_mu/Data/'+h5_in,'r') as hf:
    for cut in cuts_signal:
        ### define train, test, val sets
        # and get numbers of events
        # dics structure: dset -> particles
        nums_dict = {'train':{}, 'test':{}, 'val':{}}
        parts_dict = {'train':{}, 'test':{}, 'val':{}}
        total_nums = {'train':{}, 'test':{}, 'val':{}}
        total_all = {'train':{}, 'test':{}, 'val':{}}
        for particle in particles:
            parts = list(hf['data/cut_'+str(cut)+'/'+particle].keys())
            # assert that we have enough data
            assert len(parts)>(num_files_to_test[particle]+num_files_to_val[particle]+num_files_to_train[particle])
            # fill dics
            parts_dict['val'][particle] = parts[-num_files_to_val[particle]:]
            parts_dict['test'][particle] = parts[-num_files_to_val[particle]-num_files_to_test[particle]:-num_files_to_val[particle]]
            parts_dict['train'][particle] = parts[:-num_files_to_val[particle]-num_files_to_test[particle]]
            for ds in dsets:
                nums_dict[ds][particle] = {}
                for part in parts_dict[ds][particle]:
                    nums_dict[ds][particle][part] = hf['data/cut_'+str(cut)+'/'+particle+'/'+part+'/data'].shape[0]
                total_nums[ds][particle] = sum(nums_dict[ds][particle].values())
        for ds in dsets:
            total_all[ds] = sum(total_nums[ds].values())
    ### normalization
    # iterate over cuts
        h5_out = 'mc_mu_baikal_norm_cut-'+str(cut)+'_bundle'+h5_out_postfix+'.h5'
        with h5.File('/home/leonov/Baikal/Cut_8_mu/Data/'+h5_out,'w') as ho:
            # normilize coordiantes
            data = hf['geometry/bundle_coordinates'][()]
            mean = np.mean(data)
            std = np.std(data)
            data -= mean
            data *= 1./std
            ho.create_dataset('geometry/bundle_coordinates', data=data, dtype='float32')
            # get mean, std for data
            mean = hf['norm_param_global/cut_'+str(cut)+'/mean'+mean_posfix][()]
            std = hf['norm_param_global/cut_'+str(cut)+'/std'+mean_posfix][()]
            # loop over train/test/val
            for ds in dsets:
                # normilize          
                n = 0
                shape = hf['data/cut_'+str(cut)+'/'+particles[0]+'/part_0/data'].shape[1:]
                data = np.zeros( np.concatenate(([total_all[ds]],shape), axis=0), dtype='float32' )
                shape_chars = hf['ev_chars/cut_'+str(cut)+'/'+particles[0]+'/part_0/data'].shape[1:]
                ev_chars = np.zeros( np.concatenate(([total_all[ds]],shape_chars), axis=0), dtype='float32' )
                ev_ids = np.full( total_all[ds], 'undefined', dtype='S16' ) 
                shape = hf['mask/cut_'+str(cut)+'/'+particles[0]+'/part_0/data'].shape[1:]
                mask = np.zeros( np.concatenate(([total_all[ds]],shape), axis=0), dtype=bool )                  
                for particle in particles:
                    for part in parts_dict[ds][particle]:
                        num = int(nums_dict[ds][particle][part])
                        data[n:n+num] = (hf['data/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()].astype('float32')-mean)/std
                        ev_chars[n:n+num] = hf['ev_chars/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()]
                        ev_ids[n:n+num] = hf['ev_ids/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()]
                        mask[n:n+num] = hf['mask/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()].astype(bool)
                        n += num
                # shuffle train set
                if ds=='train':
                    idxs_shuffled = rd.sample(range(total_all[ds]), k=total_all[ds])
                else:
                    idxs_shuffled = np.arange(0, total_all[ds])        
                ho.create_dataset(ds+'/data', data=data[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/ev_chars', data=ev_chars[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/ev_ids', data=ev_ids[idxs_shuffled])
                ho.create_dataset(ds+'/mask', data=mask[idxs_shuffled], dtype='float32')
                del data
                del ev_chars
                del ev_ids
                # write labels, mask, coordinates
                shape = hf['labels/cut_'+str(cut)+'/'+particles[0]+'/part_0/data'].shape[1:]
                labels_full = np.zeros( np.concatenate(([total_all[ds]],shape), axis=0), dtype='float16' )
                labels_signal_noise = np.zeros( np.concatenate(([total_all[ds]],shape,[2]), axis=0), dtype='float16' )
                labels_particle = np.zeros( np.concatenate(([total_all[ds]],[2]), axis=0), dtype='float16' )
                labels_track_type = np.zeros( np.concatenate(([total_all[ds]],shape,[3]), axis=0), dtype='float16' )
                labels_num_mu = np.zeros( total_all[ds], dtype='float32' )
                labels_one_mu = np.zeros( np.concatenate(([total_all[ds]],[2]), axis=0), dtype='float32' )
                n = 0
                m = np.eye(2)
                mm = np.eye(3)
                for i,particle in enumerate(particles):
                    for part in parts_dict[ds][particle]:
                        num = int(nums_dict[ds][particle][part])
                        part_labels = hf['labels/cut_'+str(cut)+'/'+particle+'/'+part+'/data'][()]
                        labels_full[n:n+num] = part_labels
                        # signal-noise labels
                        idxs_pos = np.nonzero( part_labels>0 )
                        idxs_neg = np.nonzero( part_labels<0 )
                        idxs_noise = np.nonzero( part_labels==0 )
                        ls_sig = np.zeros( np.concatenate((part_labels.shape,[2]), axis=0), dtype='float16'  )
                        ls_sig[idxs_pos] = m[0]
                        ls_sig[idxs_neg] = m[0]
                        ls_sig[idxs_noise] = m[1]
                        labels_signal_noise[n:n+num] = ls_sig                   
                        # track lables
                        ls_sig = np.zeros( np.concatenate((part_labels.shape,[3]), axis=0), dtype='float16'  )
                        ls_sig[idxs_pos] = mm[0]
                        ls_sig[idxs_neg] = mm[1]
                        ls_sig[idxs_noise] = mm[2]
                        labels_track_type[n:n+num] = ls_sig
                        # particle labels
                        labels_particle[n:n+num] = np.tile( m[i], [num,1] )
                        # number of muons in group labels
                        nums_mu = np.reshape( part_labels, (part_labels.shape[0],-1) ) % 1000
                        labels_num_mu[n:n+num] = np.array([ len(np.unique(l))-1 for l in nums_mu ])
                        labels_one_mu[n:n+num] = np.where( np.expand_dims(labels_num_mu[n:n+num],axis=-1)==1,m[0],m[1])                  
                        n += num
                ho.create_dataset(ds+'/labels_full', data=labels_full[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/labels_signal_noise', data=labels_signal_noise[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/labels_track_type', data=labels_track_type[idxs_shuffled])
                ho.create_dataset(ds+'/labels_particle', data=labels_particle[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/labels_num_mu', data=labels_num_mu[idxs_shuffled], dtype='float32')
                ho.create_dataset(ds+'/labels_one_mu', data=labels_one_mu[idxs_shuffled], dtype='float32')
                # write equal weights
                #if ds=='train':
                    # l_mask: 0 for auxillary, 1 for for noise, 2 for signal
                    #l_mask = np.where( labels_signal_noise[...,0]==1, 2, mask )
                    #n_noise = len(np.nonzero(l_mask==1)[0])
                    #n_signal = len(np.nonzero(l_mask==2)[0])
                    #avg = (n_noise+n_signal)/2
                    #w_n = avg/n_noise
                    #w_s = avg/n_signal
                    #ho.create_dataset('weights/noise-signal', data=[w_n,w_s], dtype='float32')